In [6]:
import os
import glob
import random

import numpy as np
import pandas as pd
import tensorflow as tf

%matplotlib inline

In [1]:
BATCH_SIZE = 32
trdata = ImageDataGenerator()
vadata = ImageDataGenerator()

traindata = trdata.flow_from_directory(directory="../input/voice-full/train",
                                       target_size=(224,224) ,shuffle=True, batch_size=BATCH_SIZE)
validata = vadata.flow_from_directory(directory="../input/voice-full/vali", 
                                      target_size=(224,224),shuffle=True, batch_size=BATCH_SIZE) 

Found 35279 images belonging to 100 classes.
Found 2203 images belonging to 100 classes.


In [2]:
d_lab = testdata.class_indices

100

In [12]:
import tensorflow.keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D , Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras import regularizers

def model_init(num_classes, shape=(224,224,3)):
    
    filters = 64
    kernel = (3,3)
    m = Sequential()
    m.add(Conv2D(filters, kernel, input_shape=shape, padding='same', activation='relu'))
    m.add(Conv2D(filters, kernel, padding='same', activation='relu'))
    m.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) 
    m.add(Dropout(0.2))
    m.add(Conv2D(128, kernel, padding='same', activation='relu'))  
    m.add(Conv2D(128, kernel, padding='same', activation='relu'))
    m.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) 
    m.add(Dropout(0.2))
    m.add(Conv2D(256, kernel, padding='same', activation='relu'))  
    m.add(Conv2D(256, kernel, padding='same', activation='relu' ))
    m.add(MaxPooling2D(pool_size=(2,2),strides=(2,2))) 
    
    m.add(Flatten())
    m.add(Dense(1024, activation='relu'))
    m.add(Dense(num_classes, activation='softmax'))
    return m

In [13]:
from tensorflow.keras.optimizers import Adam

model = model_init(len(d_lab)) 
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss=tensorflow.keras.losses.categorical_crossentropy, metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)      

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("voice_model.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')

In [2]:
#IMAGE_SIZE = [244, 244]
hist = model.fit(
    #get_training_dataset(),
    traindata,
    steps_per_epoch=traindata.samples//traindata.batch_size,
    #steps_per_epoch=BATCH_SIZE,
    epochs=20,
    validation_data=validata, 
    validation_steps=validata.samples//validata.batch_size,
    #validation_steps=BATCH_SIZE,
    verbose=1,
    callbacks=[checkpoint,early])

NameError: name 'model' is not defined

In [ ]:
#plt.plot(hist.history["accuracy"])
#plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'][1:])
plt.plot(hist.history['val_loss'][1:])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()